In [1]:
import pandas as pd
from datetime import datetime, timedelta

def load_and_cleaned_Daily_Sell_out():
    base_path = r'T:\SCM Data\Sale Out\Daily_SO\sellout_past30D.xlsx'
    save_path = r'D:\Data for Stock Report\appended_cleaned_SellOut.xlsx'

    # Define yesterday's date
    yesterday = datetime.now().replace(hour=0, minute=0, second=0, microsecond=0) - timedelta(days=1)
    last_30D = yesterday - timedelta(days=31)

    print(f"The Report contains data Sell-Out from {last_30D} to {yesterday}")

    # Calculate date for pivoting
    today = datetime.today().replace(hour=0, minute=0, second=0, microsecond=0) 
    last_7d = today - timedelta(days=7)
    print(f"Last 7 days = {last_7d}")

    df = pd.read_excel(base_path, header=0)

    print(f"Loaded file: {base_path}")

    # Manually rename columns for the current file
    df.columns = ['Calendar Day',
                  'Product Group',
                  'DIVISION_NAME',
                  'SUB_BUSINESS',
                  'BUSINESS',
                  'Category',
                  'Sub Category',
                  'Supplier',
                  'Supplier Name',
                  'CJ_Item',
                  'Product Name',
                  'Size',
                  'Sales Store',
                  'POG Store',
                  'Base Unit',
                  'Count Asstortment Exc.B Branch',
                  'First Sales Date',
                  'Brand',
                  'Brand Name',
                  'Status',
                  'Normal Selling Price',
                  'Normal Selling Price - Start Date',
                  'Normal Selling Price - End Date',
                  'EAN/UPC',
                  'Sales unit',
                  'Total_SellOut_Qty',
                  'Total_Sellout_Amt',
                  'Normal Sale/Base QTY (TY)',
                  'Normal Sale/AMT (TY) Exc.VAT',
                  'Promotion Sale/Base QTY (TY)',
                  'Promotion Sale/AMT (TY) Exc.VAT']


    # Convert data to date time type
    df['Calendar Day'] = pd.to_datetime(df['Calendar Day'], format='%d.%m.%Y', errors='coerce')
    df['First Sales Date'] = pd.to_datetime(df['First Sales Date'], format='%d.%m.%Y', errors='coerce')

    # Check for missing values in 'Calendar Day' where other specified columns have data
    missing_dates = df[df['Calendar Day'].isna() & df[['Supplier', 'Product Name']].notna().any(axis=1)]

    if not missing_dates.empty:
        print("There are missing dates in 'Calendar Day' where other columns have data:")
        print(missing_dates)

    # Filter All data except A-Home and UNO in column Product Group
    filter_df_data = ['A-HOME','LIFESTYLE']
    df = df[~df['Product Group'].isin(filter_df_data)]
    print(f"Remove Product Group >> {filter_df_data} out of the file")

    # Create a pivot table
    pivot_first_selldate = df.pivot_table(
        index='CJ_Item',
        values ='First Sales Date',
        aggfunc='min'
    ).reset_index()

    # Pivot Sales Last 30D
    pivot_sales_last30d = df.pivot_table(
        index='CJ_Item',
        values='Total_SellOut_Qty',
        aggfunc='sum'
    ).reset_index().rename(columns={'Total_SellOut_Qty':'SO_Qty_last30D'})

    # Pivot Sales Last 7D
    pivot_sales_last7d = df[df['Calendar Day'] >= last_7d].pivot_table(
        index='CJ_Item',
        values='Total_SellOut_Qty',
        aggfunc='sum',
        fill_value=0
    ).reset_index().rename(columns={'Total_SellOut_Qty':'SO_Qty_last7D'})
    
    merged_pivot = pivot_first_selldate.merge(pivot_sales_last30d,on='CJ_Item',how='left')
    merged_pivot = merged_pivot.merge(pivot_sales_last7d,on='CJ_Item',how='left')
    merged_pivot = merged_pivot.fillna(0)

    # Write both final_data and pivot_data to the same Excel file in different sheets
    with pd.ExcelWriter(save_path, mode='w') as writer:
        df.to_excel(writer, sheet_name='Daily SO', index=False)
        merged_pivot.to_excel(writer, sheet_name='Pivot SO', index=False)

    print(f"Pivot data has been created and saved.")
    print(f"Sell out data has been cleaned and saved to: {save_path}.")

# Run the function
result = load_and_cleaned_Daily_Sell_out()

The Report contains data Sell-Out from 2024-11-17 00:00:00 to 2024-12-18 00:00:00
Last 7 days = 2024-12-12 00:00:00
Loaded file: T:\SCM Data\Sale Out\Daily_SO\sellout_past30D.xlsx
Remove Product Group >> ['A-HOME', 'LIFESTYLE'] out of the file
Pivot data has been created and saved.
Sell out data has been cleaned and saved to: D:\Data for Stock Report\appended_cleaned_SellOut.xlsx.
